In [12]:
import os
import pandas as pd
import numpy as np
import faiss
print(os.getcwd())

C:\Users\Arshraj\Desktop\GitHub\FAISS-ReviewExtraction


In [25]:
df=pd.read_csv("data\playstore_reviews.csv")
df=df[:200].copy()
reviews=df["content"].to_list()
df.info()
df.to_csv("faiss-index-df.csv")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   reviewId              200 non-null    object
 1   userName              200 non-null    object
 2   userImage             200 non-null    object
 3   content               200 non-null    object
 4   score                 200 non-null    int64 
 5   thumbsUpCount         200 non-null    int64 
 6   reviewCreatedVersion  196 non-null    object
 7   at                    200 non-null    object
 8   replyContent          1 non-null      object
 9   repliedAt             1 non-null      object
 10  appVersion            196 non-null    object
dtypes: int64(2), object(9)
memory usage: 17.3+ KB


In [27]:
from sentence_transformers import SentenceTransformer
# initialize sentence transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")
# create sentence embeddings
sentence_embeddings = model.encode(reviews,show_progress_bar=True)

C:\Users\Arshraj\anaconda3\envs\myenv1\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Batches:   0%|          | 0/7 [00:00<?, ?it/s]

In [23]:
def create_faiss_index(embeddings):
    
    # Step 1: Normalize the embeddings using FAISS's L2 normalization
    faiss.normalize_L2(embeddings)  # In-place normalization
    
    # Step 2: Create a FAISS index with IndexFlatIP (Inner Product)
    index = faiss.IndexFlatIP(embeddings.shape[1])
    
    # Step 3: Add normalized embeddings to the index
    index.add(embeddings)
    
    return index
faiss_index=create_faiss_index(sentence_embeddings)

In [24]:
faiss.write_index(faiss_index, "playstore_reviews.index")